In [ ]:
import numpy as np
import xarray as xr
import json
import pandas as pd
from datetime import datetime, timedelta
import os
import logging

logging.basicConfig(level=logging.INFO)

def dict_l(data, fm):
    try:
        # Validate filename
       
        
        # Convert data to dictionary
        logging.info("Converting data to dictionary...")
        data_dict = data.squeeze().to_dict(data='list', encoding=False)
        
        # Helper function to recursively round numeric values
        def round_numbers(obj):
            if isinstance(obj, dict):
                return {k: round_numbers(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [round_numbers(item) for item in obj]
            elif isinstance(obj, (float, np.float32, np.float64)):
                return round(float(obj), 2)  # Round floats to 2 decimal places
            elif isinstance(obj, (int, np.int32, np.int64)):
                return int(obj)  # Keep integers as-is
            else:
                return obj  # Leave other types unchanged
        
        # Apply rounding to the dictionary
        logging.info("Rounding numeric values to 2 decimal places...")
        rounded_data_dict = round_numbers(data_dict)
        
        # Helper function to handle non-serializable types
        def convert_types(obj):
            if isinstance(obj, dict):
                return {k: convert_types(v) for k, v in obj.items()}
            elif isinstance(obj, list):
                return [convert_types(item) for item in obj]
            elif isinstance(obj, (np.datetime64, pd.Timestamp, datetime)):
                return obj.isoformat()
            elif isinstance(obj, timedelta):
                return obj.total_seconds()
            elif isinstance(obj, np.ndarray):
                return obj.tolist()
            elif isinstance(obj, (np.float32, np.float64)):
                return float(obj)
            elif isinstance(obj, (np.int32, np.int64)):
                return int(obj)
            raise TypeError(f"Type {type(obj)} not serializable")
        
        # Save to JSON file
        file = os.path.join(os.getcwd(), f"{fm}.json")
        logging.info(f"Saving data to {file}...")
        with open(file, 'w') as json_file:
            json.dump(rounded_data_dict, json_file, indent=4, default=convert_types)
        
        logging.info("Data successfully saved.")
    
    except Exception as e:
        logging.error(f"An error occurred: {e}")
        raise

In [ ]:
from datetime import date

def get_today_date():
    return date.today().strftime("%Y%m%d")

# Example usage:
today_date = get_today_date()
print(today_date)


In [ ]:
url=f"http://nomads.ncep.noaa.gov:80/dods/gfs_0p25_1hr/gfs{today_date}/gfs_0p25_1hr_18z"

In [ ]:
ds =xr.open_dataset(url).sel(lon=slice(60,100),lat=slice(0,40))


In [ ]:
sfcx=ds['apcpsfc'].sel(lon=slice(60,100),lat=slice(0,40))#.resample(time='3H').mean()
dt = sfcx.time


In [ ]:
sfcx1=sfcx.fillna(0)
sfcx=sfcx1.diff(dim='time').dropna(dim='time')

rf=ds['apcpsfc'][:72].round(2)
temp=ds['tmpsfc'].round(2)
mslp=ds['pressfc'].round(2)/100
cloud=ds['tcdcaveclm'].round(2)

In [ ]:
dict_l(rf[:96],"rf_tot_nasa")
dict_l(temp[:96],"temp_tot_nasa")
dict_l(mslp[:96],"mslp_tot_nasa")
dict_l(cloud[:96],"cloud_tot_nasa")
